Get Memory Allocations from their Memory Clerks

In [1]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;

DECLARE @sqlcmd NVARCHAR(max), @params NVARCHAR(600)
DECLARE @sqlmajorver int, @xtp bit

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);

SET @sqlcmd = N'SELECT ''Memory_checks'' AS [Category], [type] AS Alloc_Type, 
' + CASE WHEN @sqlmajorver < 11 THEN 'SUM(single_pages_kb + multi_pages_kb + virtual_memory_committed_kb + shared_memory_committed_kb + awe_allocated_kb) AS Alloc_Mem_KB'
	ELSE 'SUM(pages_kb + virtual_memory_committed_kb + shared_memory_committed_kb + awe_allocated_kb) AS Alloc_Mem_KB' END + '
FROM sys.dm_os_memory_clerks 
WHERE type IN (''CACHESTORE_COLUMNSTOREOBJECTPOOL'',''CACHESTORE_CLRPROC'',''CACHESTORE_OBJCP'',''CACHESTORE_PHDR'',''CACHESTORE_SQLCP'',''CACHESTORE_TEMPTABLES'',
''MEMORYCLERK_SQLBUFFERPOOL'',''MEMORYCLERK_SQLCLR'',''MEMORYCLERK_SQLGENERAL'',''MEMORYCLERK_SQLLOGPOOL'',''MEMORYCLERK_SQLOPTIMIZER'',
''MEMORYCLERK_SQLQUERYCOMPILE'',''MEMORYCLERK_SQLQUERYEXEC'',''MEMORYCLERK_SQLQUERYPLAN'',''MEMORYCLERK_SQLSTORENG'',''MEMORYCLERK_XTP'',
''OBJECTSTORE_LOCK_MANAGER'',''OBJECTSTORE_SNI_PACKET'',''USERSTORE_DBMETADATA'',''USERSTORE_OBJPERM'')
GROUP BY [type]
UNION ALL
SELECT ''Memory_checks'' AS [Category], ''Others'' AS Alloc_Type, 
' + CASE WHEN @sqlmajorver < 11 THEN 'SUM(single_pages_kb + multi_pages_kb + virtual_memory_committed_kb + shared_memory_committed_kb) AS Alloc_Mem_KB'
	ELSE 'SUM(pages_kb + virtual_memory_committed_kb + shared_memory_committed_kb) AS Alloc_Mem_KB' END + '
FROM sys.dm_os_memory_clerks 
WHERE type NOT IN (''CACHESTORE_COLUMNSTOREOBJECTPOOL'',''CACHESTORE_CLRPROC'',''CACHESTORE_OBJCP'',''CACHESTORE_PHDR'',''CACHESTORE_SQLCP'',''CACHESTORE_TEMPTABLES'',
''MEMORYCLERK_SQLBUFFERPOOL'',''MEMORYCLERK_SQLCLR'',''MEMORYCLERK_SQLGENERAL'',''MEMORYCLERK_SQLLOGPOOL'',''MEMORYCLERK_SQLOPTIMIZER'',
''MEMORYCLERK_SQLQUERYCOMPILE'',''MEMORYCLERK_SQLQUERYEXEC'',''MEMORYCLERK_SQLQUERYPLAN'',''MEMORYCLERK_SQLSTORENG'',''MEMORYCLERK_XTP'',
''OBJECTSTORE_LOCK_MANAGER'',''OBJECTSTORE_SNI_PACKET'',''USERSTORE_DBMETADATA'',''USERSTORE_OBJPERM'')
ORDER BY Alloc_Mem_KB DESC'
EXECUTE sp_executesql @sqlcmd;

IF @sqlmajorver >= 12
BEGIN
	SET @sqlcmd = N'SELECT @xtpOUT = COUNT(*) FROM sys.dm_db_xtp_memory_consumers';
	SET @params = N'@xtpOUT int OUTPUT';
	EXECUTE sp_executesql @sqlcmd, @params, @xtpOUT = @xtp OUTPUT;
	
	IF @xtp > 0
	BEGIN
		SET @sqlcmd = N'SELECT ''Memory_checks'' AS [Category], ''InMemory_Consumers'' AS Alloc_Type, 
OBJECT_NAME([object_id]) AS [Object_Name], memory_consumer_type_desc, [object_id], index_id, 
allocated_bytes/(1024*1024) AS Allocated_MB, used_bytes/(1024*1024) AS Used_MB, 
CASE WHEN used_bytes IS NULL THEN ''used_bytes_is_varheap_only'' ELSE '''' END AS [Comment]
FROM sys.dm_db_xtp_memory_consumers
WHERE [object_id] > 0
ORDER BY Allocated_MB DESC' -- Only user objects; system objects are negative numbers
		EXECUTE sp_executesql @sqlcmd;

		SET @sqlcmd = N'SELECT ''Memory_checks'' AS [Category], ''InMemory_Alloc'' AS Alloc_Type, 
SUM(allocated_bytes)/(1024*1024) AS total_allocated_MB, SUM(used_bytes)/(1024*1024) AS total_used_MB
FROM sys.dm_db_xtp_memory_consumers
ORDER BY total_allocated_MB DESC'
		EXECUTE sp_executesql @sqlcmd;
	END;
END;

Commands completed successfully.

Total execution time: 00:00:00.021

Category,Alloc_Type,Alloc_Mem_KB
Memory_checks,MEMORYCLERK_SQLBUFFERPOOL,99080
Memory_checks,Others,97988
Memory_checks,OBJECTSTORE_LOCK_MANAGER,66368
Memory_checks,MEMORYCLERK_SQLSTORENG,54040
Memory_checks,MEMORYCLERK_SQLCLR,30712
Memory_checks,MEMORYCLERK_SQLGENERAL,17192
Memory_checks,MEMORYCLERK_SQLQUERYPLAN,4480
Memory_checks,CACHESTORE_SQLCP,3520
Memory_checks,MEMORYCLERK_SQLLOGPOOL,2672
Memory_checks,CACHESTORE_PHDR,2472
